<a href="https://colab.research.google.com/github/nivea0284-coder/cartoprint-/blob/main/C%C3%B3pia_de_C%C3%B3pia_de_C%C3%B3pia_de_Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import folium
import math
from IPython.display import display
from google.colab import files


# ==========================================
# CLASSE BASE (HERANÇA)
# ==========================================
class SistemaBase:

    def exportar(self, nome):
        print("Método exportar da classe base.")

    def importar(self, nome):
        print("Método importar da classe base.")


# ==========================================
# CLASSE PRINCIPAL
# ==========================================
class CartoPoint(SistemaBase):

    def __init__(self):
        self.dados = pd.DataFrame(columns=["Nome", "Latitude", "Longitude"])


    def adicionar_ponto(self, nome, lat, lon):
        novo = {
            "Nome": nome,
            "Latitude": float(lat),
            "Longitude": float(lon)
        }
        self.dados = pd.concat(
            [self.dados, pd.DataFrame([novo])],
            ignore_index=True
        )
        print("Ponto adicionado com sucesso!")


    def mostrar_dados(self):
        if self.dados.empty:
            print("Nenhum dado cadastrado.")
        else:
            display(self.dados)


    def gerar_mapa(self):
        if self.dados.empty:
            print("Nenhum dado cadastrado.")
            return None

        mapa = folium.Map(
            location=[
                self.dados["Latitude"].mean(),
                self.dados["Longitude"].mean()
            ],
            zoom_start=5
        )

        for _, linha in self.dados.iterrows():
            folium.Marker(
                [linha["Latitude"], linha["Longitude"]],
                popup=linha["Nome"]
            ).add_to(mapa)

        return mapa


    def calcular_distancia(self, i1, i2):
        if i1 not in self.dados.index or i2 not in self.dados.index:
            print("Índice inválido.")
            return

        lat1 = math.radians(self.dados.loc[i1, "Latitude"])
        lon1 = math.radians(self.dados.loc[i1, "Longitude"])
        lat2 = math.radians(self.dados.loc[i2, "Latitude"])
        lon2 = math.radians(self.dados.loc[i2, "Longitude"])

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = math.sin(dlat/2)**2 + \
            math.cos(lat1) * math.cos(lat2) * \
            math.sin(dlon/2)**2

        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distancia = 6371 * c

        print(f"Distância: {distancia:.3f} km")


    def excluir_por_indice(self, indice):
        if indice in self.dados.index:
            nome = self.dados.loc[indice, "Nome"]
            self.dados = self.dados.drop(indice).reset_index(drop=True)
            print(f"Ponto '{nome}' excluído!")
        else:
            print("Índice inválido.")


    def excluir_por_nome(self, nome):
        if nome in self.dados["Nome"].values:
            self.dados = self.dados[self.dados["Nome"] != nome].reset_index(drop=True)
            print(f"Ponto '{nome}' excluído!")
        else:
            print("Nome não encontrado.")


    #  POLIMORFISMO
    def exportar(self, nome):
        if nome.endswith(".csv"):
            self.exportar_csv(nome)
        elif nome.endswith(".json"):
            self.exportar_json(nome)
        else:
            print("Formato não suportado.")


    def importar(self, nome):
        if nome.endswith(".csv"):
            self.importar_csv(nome)
        elif nome.endswith(".json"):
            self.importar_json(nome)
        else:
            print("Formato não suportado.")


    def exportar_csv(self, nome="dados.csv"):
        if self.dados.empty:
            print("Nenhum dado para exportar.")
            return
        self.dados.to_csv(nome, index=False)
        files.download(nome)
        print("Download CSV iniciado!")


    def exportar_json(self, nome="dados.json"):
        if self.dados.empty:
            print("Nenhum dado para exportar.")
            return
        self.dados.to_json(nome, orient="records", indent=4)
        files.download(nome)
        print("Download JSON iniciado!")


    def importar_csv(self, nome):
        self.dados = pd.read_csv(nome)
        print("CSV importado com sucesso!")
        display(self.dados)


    def importar_json(self, nome):
        self.dados = pd.read_json(nome)
        print("JSON importado com sucesso!")
        display(self.dados)


# ==========================================
# MENU INTERATIVO
# ==========================================

carto = CartoPoint()

executando = True

while executando:
    print("\n===== CARTOPOINT =====")
    print("1 - Adicionar ponto")
    print("2 - Mostrar dados")
    print("3 - Gerar mapa")
    print("4 - Calcular distância")
    print("5 - Excluir por índice")
    print("6 - Excluir por nome")
    print("7 - Exportar CSV")
    print("8 - Exportar JSON")
    print("9 - Importar CSV")
    print("10 - Importar JSON")
    print("11 - Sair")

    opcao = input("Escolha: ")

    if opcao == "1":
        nome = input("Nome: ")
        lat = input("Latitude: ").replace(",", ".")
        lon = input("Longitude: ").replace(",", ".")
        carto.adicionar_ponto(nome, lat, lon)

    elif opcao == "2":
        carto.mostrar_dados()

    elif opcao == "3":
        display(carto.gerar_mapa())

    elif opcao == "4":
        carto.mostrar_dados()
        i1 = int(input("Índice 1: "))
        i2 = int(input("Índice 2: "))
        carto.calcular_distancia(i1, i2)

    elif opcao == "5":
        carto.mostrar_dados()
        indice = int(input("Índice para excluir: "))
        carto.excluir_por_indice(indice)

    elif opcao == "6":
        nome = input("Nome do ponto: ")
        carto.excluir_por_nome(nome)

    elif opcao == "7":
        carto.exportar("dados.csv")

    elif opcao == "8":
        carto.exportar("dados.json")

    elif opcao == "9":
        nome = input("Nome do arquivo CSV: ")
        carto.importar(nome)

    elif opcao == "10":
        nome = input("Nome do arquivo JSON: ")
        carto.importar(nome)

    elif opcao == "11":
        print("Encerrando sistema...")
        executando = False

    else:
        print("Opção inválida.")


===== CARTOPOINT =====
1 - Adicionar ponto
2 - Mostrar dados
3 - Gerar mapa
4 - Calcular distância
5 - Excluir por índice
6 - Excluir por nome
7 - Exportar CSV
8 - Exportar JSON
9 - Importar CSV
10 - Importar JSON
11 - Sair
